In [0]:
import pandas as pd
import numpy as np
import sklearn
import warnings
warnings.filterwarnings("ignore")

In [4]:
titanic_data = pd.read_csv("titanic.csv")
titanic_data.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [5]:
titanic_data.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [6]:
titanic_data.isnull().sum() #sum gives total no. of null values in every column

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [7]:
titanic_data.info() #information of dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [0]:
titanic_data.drop("Cabin",axis=1,inplace=True)

In [0]:
#fill age values
titanic_data['Age'].fillna((titanic_data['Age'].mean()), inplace=True)

In [0]:
titanic_data.dropna(inplace=True)

In [11]:
#to create dummies 
sex = pd.get_dummies(titanic_data["Sex"],drop_first=True)
sex.head(2)

,male
0,1
1,0


In [12]:
Pcl = pd.get_dummies(titanic_data["Pclass"],drop_first=True)
Pcl.head(2)

,2,3
0,0,1
1,0,0


In [13]:
embark = pd.get_dummies(titanic_data["Embarked"],drop_first=True)
embark.head(2)

,Q,S
0,0,1
1,0,0


In [14]:
titanic_data = pd.concat([titanic_data,sex,Pcl,embark],axis = 1) #concatinating the new data which are all integers
titanic_data.drop(['Sex','Embarked','Pclass','Name','Ticket','PassengerId'],inplace=True,axis=1) #eliminating the coloumns with strings or characters as they are converted into int above
titanic_data.head(2)

,Survived,Age,SibSp,Parch,Fare,male,2,3,Q,S
0,0,22.0,1,0,7.2500,1,0,1,0,1
1,1,38.0,1,0,71.2833,0,0,0,0,0


In [0]:
#Training and testing
X = titanic_data.drop("Survived",axis=1)
y = titanic_data["Survived"]
from sklearn.model_selection import train_test_split

In [16]:
#Splitting
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=1)
from sklearn.linear_model import LogisticRegression
logmodel = LogisticRegression()
logmodel.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [17]:
predictions = logmodel.predict(X_test)
#metrics
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,predictions)*100)

83.14606741573034


In [18]:
!pip install flask-ngrok

In [19]:
predictions = logmodel.predict([[22.0,1,0,7.2500,1,0,1,0,1]]) #after execution of this goto 39th cell
predictions

array([0])

In [24]:
from flask_ngrok import run_with_ngrok
from flask import Flask,jsonify
app = Flask(__name__)
run_with_ngrok(app) #starts when app is running
@app.route("/<int:Age>/<int:SibSp>/<int:Parch>/<float:Fare>/<Gender>/<int:PClass>/<Place>")
def home(Age,SibSp,Parch,Fare,Gender,PClass,Place):
  p = []
  p += [Age,SibSp,Parch,Fare]
  if Gender.casefold() == "m":
    p+=[1]
  else:
    p+=[0]
  if PClass == 2:
    p+=[1,0]
  elif PClass == 3:
    p+=[0,1]
  else:
    p+=[0,0]
  if Place.casefold() == "queenstown":
    p+=[1,0]
  elif Place.casefold() == "southampton":
    p+=[0,1]
  else:
    p+=[0,0]
  arr = np.array([p])
  predict = logmodel.predict(arr)
  if predict == [1]:
    result = {'person':'Survived'}
  else:
    result = {'person':'NotSurvived'}
  
  return jsonify(result)
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://f8508a10.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [06/May/2020 16:01:04] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [06/May/2020 16:01:04] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [06/May/2020 16:01:04] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [06/May/2020 16:01:09] "GET /22/1/0/7.25/M/3/southampton HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 16:01:09] "GET /22/1/0/7.25/M/3/southampton HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 16:01:55] "GET /22/1/0/7.25/m/3/Southampton HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 16:01:57] "GET /22/1/0/7.25/m/3/Southampton HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 16:02:49] "GET /35/1/0/7.25/m/3/Southampton HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 16:02:54] "GET /35/1/0/7.25/m/3/Southampton HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 16:02:58] "GET /20/0/1/7.25/m/3/Southampton HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 16:03:14] "GET /20/0/1/7.25/m/3/Southampton HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 16:05:17] "GET /27/0/2/11.1333/f/3/Southampton HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2